In [10]:
import json
import os
from openai import OpenAI

# Requires an API key to be set as an environment variable! :$it's a secret$: 
# Create an issue on the repository if you need help with that (or just ask ChatGPT!)
# All student responses from the survey are stored in this file
with open('all_responses.json', 'r') as file:
    data = json.load(file)
    all_responses = data['responses']

# Example ratings for few shot learning of the following open-ended question in the survey:
# "what do you know about neural networks (write in one sentence without looking it up)?"
system_prompt = "In the pre-survey and the post-survey, the question is: ``What do you know about neural networks (write in one sentence without looking it up)?'' Students have responded to this question. The task is to score each response based on the following criteria on ``Fidelity''. Criteria Title: Technical Depth & Accuracy. Description: Evaluates the richness and correctness of the technical details provided in the answer"
example_responses = [
    "Neural networks are a series of hidden layers (Conv2D, BatchNorm, MaxPooling, etc.) that each contain a certain number of trainable parameters (neurons), which are updated by using gradient descent after each iteration when the loss is calculated",
    "Neural networks is the structure behind artificial intelligence that make it able to ``think'' and operate the way it does.",
    "A neural network is a way for AI to simulate the human brain through interconnected nodes with many different hidden layers."
]
example_ratings = [0.9, 0.2, 0.6]

# The classifier function
def rate_response(response):
    prompt = "\n".join(
        [system_prompt] + 
        [f"Input text: '{resp}'. Rating: {rate}" for resp, rate in zip(example_responses, example_ratings)]
    )
    
    gpt_output = OpenAI().chat.completions.create(
        model="gpt-3.5-turbo", # choose the GPT model you want to use
        messages=[
            {
                "role": "system",
                "content": prompt,
            },
            {
                "role":"user",
                "content":f"Input text: {response}. Rating:"
            }
        ],
        max_tokens=5 # To limit the model output
    )
    return gpt_output

# Classify each comment and store
all_ratings = []
all_rated_responses = {}

for response in all_responses:
    output = rate_response(response)
    rating_float = output.choices[0].message.content
    all_ratings.append(rating_float)
    all_rated_responses[response] = rating_float
    
# Save a new JSON file
with open('rated_responses.json', 'w') as file:
    json.dump(all_rated_responses, file, indent=4)

print("Responses have been rated and saved to rated_responses.json.")


ChatCompletionMessage(content='0.9', role='assistant', function_call=None, tool_calls=None)
ChatCompletionMessage(content='0.7', role='assistant', function_call=None, tool_calls=None)
Responses have been rated and saved to rated_responses.json.
